# Day 11: Care Package

In [5]:
from collections import defaultdict

In [6]:
class Instruction:
    def __init__(self, opcode, length, param_locations, param_values, memory):
        self.opcode = opcode
        self.length = length
        self.param_locations = param_locations
        self.param_values = param_values
        self.memory = memory
        
    def get_val(self, i):
        return self.param_values[i]
    
    def set_val(self, i, val):
        self.memory[self.param_locations[i]] = val

In [7]:
class Add(Instruction):
    def call(self):
        self.set_val(2, self.get_val(0) + self.get_val(1))

In [8]:
class Multiply(Instruction):
    def call(self):
        self.set_val(2, self.get_val(0) * self.get_val(1))

In [9]:
class Input(Instruction):
    def call(self, input_value):
        self.set_val(0, input_value)

In [10]:
class Output(Instruction):
    def call(self):
        return self.get_val(0)

In [11]:
class JumpIfTrue(Instruction):
    def call(self):
        return self.get_val(1) if self.get_val(0) else None

In [12]:
class JumpIfFalse(Instruction):
    def call(self):
        return self.get_val(1) if not self.get_val(0) else None

In [13]:
class LessThan(Instruction):
    def call(self):
        val = 1 if self.get_val(0) < self.get_val(1) else 0
        self.set_val(2, val)

In [14]:
class Equals(Instruction):
    def call(self):
        val = 1 if self.get_val(0) == self.get_val(1) else 0
        self.set_val(2, val)

In [15]:
class RelativeBaseOffset(Instruction):
    def call(self):
        return self.get_val(0)

In [16]:
class Halt(Instruction):
    pass

In [17]:
class IntcodeComputer:
    # Instruction opcodes
    ADD = 1
    MULTIPLY = 2
    INPUT = 3
    OUTPUT = 4
    JUMP_IF_TRUE = 5
    JUMP_IF_FALSE = 6
    LESS_THAN = 7
    EQUALS = 8
    RELATIVE_BASE_OFFSET = 9
    HALT = 99
    
    LENGTHS = {
        ADD: 4,
        MULTIPLY: 4,
        INPUT: 2,
        OUTPUT: 2,
        JUMP_IF_TRUE: 3,
        JUMP_IF_FALSE: 3,
        LESS_THAN: 4,
        EQUALS: 4,
        RELATIVE_BASE_OFFSET: 2,
        HALT: 1,
    }
    
    INSTRUCTIONS = {
        ADD: Add,
        MULTIPLY: Multiply,
        INPUT: Input,
        OUTPUT: Output,
        JUMP_IF_TRUE: JumpIfTrue,
        JUMP_IF_FALSE: JumpIfFalse,
        LESS_THAN: LessThan,
        EQUALS: Equals,
        RELATIVE_BASE_OFFSET: RelativeBaseOffset,
        HALT: Halt,
    }
    
    POSITION_MODE = '0'
    IMMEDIATE_MODE = '1'
    RELATIVE_MODE = '2'
    
    def __init__(self, program):
        self.memory = program.copy()
        self.memory_pointer = 0
        self.relative_base = 0
        self.inputs = []
        
    def create_instruction(self):
        op = str(self.memory[self.memory_pointer])
        opcode = int(op[-2:])    
        length = self.LENGTHS[opcode]
        args = [self.memory[self.memory_pointer + i] for i in range(1, length)]
        modes = op[:-2].zfill(length - 1)[::-1]
        locs = []
        vals = []
        for arg, mode in zip(args, modes):
            if mode == self.POSITION_MODE: 
                locs.append(arg)
                vals.append(self.memory[arg])
            elif mode == self.IMMEDIATE_MODE:
                locs.append(None)
                vals.append(arg)
            elif mode == self.RELATIVE_MODE:
                locs.append(arg + self.relative_base)
                vals.append(self.memory[arg + self.relative_base])

        return self.INSTRUCTIONS[opcode](opcode, length, locs, vals, self.memory)
    
    def queue_input(self, input_value):
        self.inputs.append(input_value)
        
    def run(self):
        while True:
            instruction = self.create_instruction()
            if instruction.opcode == self.HALT:
                break
            elif instruction.opcode == self.INPUT:
                value = instruction.call(self.inputs.pop(0))
            else:
                value = instruction.call()
            
            self.memory_pointer += instruction.length
            if instruction.opcode in [self.JUMP_IF_TRUE, self.JUMP_IF_FALSE] and value is not None:
                self.memory_pointer = value
            elif instruction.opcode == self.RELATIVE_BASE_OFFSET:
                self.relative_base += value
            elif instruction.opcode == self.OUTPUT:
                yield value

    @staticmethod
    def to_program(data):
        return defaultdict(int, enumerate(data.copy()))

## Part 1

In [19]:
def build_tiles(computer):
    tiles = defaultdict(int)
    x_coord, y_coord = [None, None]
    for i, output in enumerate(computer.run()):
        if i % 3 == 0:
            x_coord = output
        elif i % 3 == 1:
            y_coord = output
        else:
            tiles[(x_coord, y_coord)] = output
    return tiles

In [20]:
with open('day_13.txt') as f:
    data = [int(token) for token in f.readlines()[0].strip().split(',')]

In [21]:
program = IntcodeComputer.to_program(data)
computer = IntcodeComputer(program)
tiles = build_tiles(computer)

In [23]:
len([coords for coords in tiles if tiles[coords] == 2])

236

## Part 2

In [27]:
EMPTY = 0
WALL = 1
BLOCK = 2
PADDLE = 3
BALL = 4

TILE_CHARS = {
    EMPTY: " ",
    WALL: "#",
    BLOCK: "X",
    PADDLE: "=",
    BALL: "O"
}

In [41]:
def print_display(tiles):
    min_x = min([loc[0] for loc in tiles])
    max_x = max([loc[0] for loc in tiles])
    min_y = min([loc[1] for loc in tiles])
    max_y = max([loc[1] for loc in tiles])
    for y in range(min_y, max_y):
        row = [TILE_CHARS[tiles[(x, y)]] for x in range(min_x, max_x + 1)]
        print("".join(row))

In [42]:
print_display(tiles)

####################################
#                                  #
# XX X    XX XXX XXXX  X X   XXXXX #
# X  XXX  X  X X  XX  X  XXX X XXX #
# XX XX X   X X  X X  XXXX XXX   X #
#    X  XX  XX X XX XXX   XXXX XXX #
# X       XXXX XXXXX XX X   XXX X  #
# X XXXXX  X   X XX  X XXXX X      #
# XX   X   XX XXX XXX XXXXXXX XXXX #
#  X XXXX  XX  X  X  XX  X XX  X X #
# XXXX X XX XXXX       XXXXX X   X #
# X   X  X  X   XXXXX X X X X  XX  #
#  X X  X X XXX X XXXXX  XXXX XXX  #
# XXXX  XX  X   X  XX  XX XX XXXXX #
# XXX  XX XX  XX   XX XXX XXXXXXXX #
#                                  #
#               O                  #
#                                  #
#                                  #
#                 =                #
#                                  #


In [75]:
def play(computer):
    score = 0
    tiles = defaultdict(int)
    x_coord, y_coord, x_ball, x_paddle = [None, None, None, None]
    computer.queue_input(0)
    for i, output in enumerate(computer.run()):
        if i % 3 == 0:
            x_coord = output
            continue
        elif i % 3 == 1:
            y_coord = output
            continue
        elif (x_coord, y_coord) == (-1, 0):
            score = output
            continue
        tiles[(x_coord, y_coord)] = output
        if output == BALL:
            if not x_paddle:
                pass
            elif x_ball < x_paddle:
                computer.queue_input(-1)
            elif x_ball > x_paddle:
                computer.queue_input(1)
            elif x_ball < x_coord:
                computer.queue_input(1)
            elif x_ball > x_coord:
                computer.queue_input(-1)
            else:
                computer.queue_input(0)
            x_ball = x_coord
        elif output == PADDLE:
            x_paddle = x_coord
#             print_display(tiles)
    return score

In [76]:
program[0] = 2
computer = IntcodeComputer(program)
play(computer)

11040